In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Tentukan jalur direktori untuk data latih, validasi, dan uji
train_data = "../Oprek/Ori_Data_Aug/train" 
val_data = "../Oprek/Ori_Data_Aug/val"
test_data = "../Oprek/Ori_Data_Aug/test"

# train_data = '../Oprek/data_split/train'
# test_data = '../Oprek/data_split/test'

In [3]:
# Applying Data normalization and Data augmentation
img_datagen = ImageDataGenerator(
    rescale=1./255
)

# validation_datagen = ImageDataGenerator(
#     # rescale=1./255
# )

train_generator = img_datagen.flow_from_directory(
    train_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(64, 64),
    class_mode="categorical"
)

validation_generator = img_datagen.flow_from_directory(
    val_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(64, 64),
    class_mode="categorical"
)

test_generator = img_datagen.flow_from_directory(
    test_data,
    batch_size=32,
    color_mode="rgb",
    target_size=(64, 64),
    class_mode="categorical"
)

Found 8400 images belonging to 20 classes.
Found 2400 images belonging to 20 classes.
Found 1200 images belonging to 20 classes.


# Model CNN tanpa Arsitekur Vgg16 & Tuning Parameter

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Membuat model CNN
model = Sequential()

# Blok 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))

# Blok 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Blok 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Mengubah hasil dari layer sebelumnya menjadi vektor
model.add(Flatten())

# Fully connected layers
model.add(Dense(100, activation='relu'))
model.add(Dense(20, activation='softmax'))  # Jumlah kelas sesuai dengan dataset Anda

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print ringkasan model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [5]:
# from keras.callbacks import Callback

# class CustomCallback(Callback):
#     def on_epoch_end(self, epoch, logs=None):
#         if logs is not None:
#             print(f"Epoch {epoch + 1} - Accuracy: {logs['accuracy']:.1%}")

# from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback
# early_stopping_callback = EarlyStopping(
    # monitor='accuracy', 
    # patience=3, 
    # min_delta=0.01, 
    # baseline=0.98, 
    # mode='max' 
    # verbose=1
# )

In [6]:
from tensorflow.keras.callbacks import Callback

class AccuracyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] >= 0.98:
            print("\nAkurasi mencapai 95%, menghentikan pelatihan!")
            self.model.stop_training = True

# Callback untuk menghentikan pelatihan
accuracy_callback = AccuracyCallback()

In [7]:
# Train the model with EarlyStopping callback
history = model.fit(
    train_generator, 
    epochs=15, 
    validation_data=validation_generator,  
    callbacks=[accuracy_callback]
)

Epoch 1/15
263/263 [==============================] - 365s 1s/step - loss: 1.2705 - accuracy: 0.6123 - val_loss: 0.8114 - val_accuracy: 0.8067
Epoch 2/15
263/263 [==============================] - 239s 906ms/step - loss: 0.1160 - accuracy: 0.9662 - val_loss: 0.8295 - val_accuracy: 0.8496
Epoch 3/15
263/263 [==============================] - ETA: 0s - loss: 0.0262 - accuracy: 0.9935
Akurasi mencapai 95%, menghentikan pelatihan!
263/263 [==============================] - 205s 779ms/step - loss: 0.0262 - accuracy: 0.9935 - val_loss: 0.7242 - val_accuracy: 0.8621
